In [ ]:
from google.colab import files
uploaded=files.upload()
filename=list(uploaded.keys())[0]
filename

Saving kakaoapikey.txt to kakaoapikey.txt


'kakaoapikey.txt'

#행안부 도로명 주소 API 활용

* api key 가져오기

In [31]:
with open('roadapikey2.txt') as file:
  road_key=file.read()

In [32]:
len(road_key)

43

URL 만들기

In [10]:
import requests
from urllib.parse import quote

In [12]:
#한글 반드시 인코딩해서 보내야한다
quote('서초구청')

'%EC%84%9C%EC%B4%88%EA%B5%AC%EC%B2%AD'

In [33]:
road_url='https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f"keyword={quote('서초구청')}&resultType=json"
url = f'{road_url}?{params1}&{params2}'

In [34]:
result=requests.get(url)
result.status_code

200

In [15]:
import json

In [ ]:
res=json.loads(result.text[1:-1])
res

res 에서 세건의 주소 (roadAddr)가져오기

In [36]:
res.keys()

dict_keys(['results'])

In [37]:
res['results'].keys()

dict_keys(['common', 'juso'])

In [38]:
len(res['results'].keys())

2

In [40]:
len(res['results']['juso'])

3

In [45]:
#'roadAddr': '서울특별시 서초구 남부순환로 2584(서초동)'
print(res['results']['juso'][0]['roadAddr'])

서울특별시 서초구 남부순환로 2584(서초동)


In [51]:
print(res['results']['juso'][0]['roadAddr'])
print(res['results']['juso'][1]['roadAddr'])
print(res['results']['juso'][2]['roadAddr'])

서울특별시 서초구 남부순환로 2584(서초동)
서울특별시 서초구 남부순환로347길 46(서초동)
서울특별시 서초구 양재대로12길 73-19(원지동)


서초구청, 강남구청, 동작구청, 용산구청, 성동구청의 도로명 주소 가져오기

In [52]:
road_url='https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f"keyword={quote('서초구청')}&resultType=json"
url = f'{road_url}?{params1}&{params2}'

In [54]:
places= '서초구청, 강남구청, 동작구청, 용산구청, 성동구청'.split(',')
addr_list=[]
for place in places:
  url = f'{road_url}?{params1}&keyword={quote(place)}&resultType=json'
  road_url='https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do'
  result=requests.get(url)
  res=json.loads(result.text[1:-1])
  addr=res['results']['juso'][0]['roadAddr']
  addr_list.append(addr)

addr_list

['서울특별시 서초구 남부순환로 2584(서초동)',
 '서울특별시 강남구 도곡로18길 7(도곡동)',
 '서울특별시 동작구 장승배기로 161(노량진동)',
 '서울특별시 용산구 백범로79길 64(청파동3가)',
 '서울특별시 성동구 고산자로 270(행당동)']

In [57]:
import pandas as pd

In [58]:
df=pd.DataFrame({'장소':places ,'도로명주소':addr_list})
df

,장소,도로명주소
0,서초구청,서울특별시 서초구 남부순환로 2584(서초동)
1,강남구청,서울특별시 강남구 도곡로18길 7(도곡동)
2,동작구청,서울특별시 동작구 장승배기로 161(노량진동)
3,용산구청,서울특별시 용산구 백범로79길 64(청파동3가)
4,성동구청,서울특별시 성동구 고산자로 270(행당동)


In [59]:
df.to_csv('서울관공서.csv',index=False)

KaKao REST API

In [76]:
with open('kakaoapikey.txt') as file:
  kakao_key=file.read()
len(kakao_key)

32

* URL 만들기

In [77]:
local_url='https://dapi.kakao.com/v2/local/search/address.json'
addr='서울특별시 서초구 남부순환로 2584(서초동)'
url=f'{local_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%84%9C%EC%B4%88%EA%B5%AC%20%EB%82%A8%EB%B6%80%EC%88%9C%ED%99%98%EB%A1%9C%202584%28%EC%84%9C%EC%B4%88%EB%8F%99%29'

In [80]:
# Header: "Authorization: KakaoAK ${REST_API_KEY}"
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [81]:
result=requests.get(url,headers=header).json()
result

{'documents': [{'address': {'address_name': '서울 서초구 서초동 1376-3',
    'b_code': '1165010800',
    'h_code': '1165052000',
    'main_address_no': '1376',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '서초구',
    'region_3depth_h_name': '서초2동',
    'region_3depth_name': '서초동',
    'sub_address_no': '3',
    'x': '127.032683002019',
    'y': '37.4836248649455'},
   'address_name': '서울 서초구 남부순환로 2584',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 서초구 남부순환로 2584',
    'building_name': '서초구청, 서초구의회',
    'main_building_no': '2584',
    'region_1depth_name': '서울',
    'region_2depth_name': '서초구',
    'region_3depth_name': '서초동',
    'road_name': '남부순환로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.032683002019',
    'y': '37.4836248649455',
    'zone_no': '06750'},
   'x': '127.032683002019',
   'y': '37.4836248649455'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [ ]:
#'x':'127.032683002019'
#'y':'37.4836248649455'

In [82]:
result['documents'][0]['x']

'127.032683002019'

In [83]:
result['documents'][0]['y']

'37.4836248649455'

In [86]:
#latitude longitude
lng=result['documents'][0]['x']
lat=result['documents'][0]['y']
lat,lng

('37.4836248649455', '127.032683002019')

* 서울 관공서의 좌표를 채우기

In [88]:
df=pd.read_csv('서울관공서.csv')
df

,장소,도로명주소
0,서초구청,서울특별시 서초구 남부순환로 2584(서초동)
1,강남구청,서울특별시 강남구 도곡로18길 7(도곡동)
2,동작구청,서울특별시 동작구 장승배기로 161(노량진동)
3,용산구청,서울특별시 용산구 백범로79길 64(청파동3가)
4,성동구청,서울특별시 성동구 고산자로 270(행당동)


In [89]:
df.index

RangeIndex(start=0, stop=5, step=1)

In [92]:
lng_list,lat_list=[],[]
for i in df.index:
  url=f'{local_url}?query={quote(df.도로명주소[i])}'
  result=requests.get(url,headers=header).json()
  lng_list.append(float(result['documents'][0]['x']))
  lat_list.append(float(result['documents'][0]['y']))

In [93]:
df['위도']=lat_list
df['경도']=lng_list

In [94]:
df

,장소,도로명주소,위도,경도
0,서초구청,서울특별시 서초구 남부순환로 2584(서초동),37.483625,127.032683
1,강남구청,서울특별시 강남구 도곡로18길 7(도곡동),37.490901,127.038442
2,동작구청,서울특별시 동작구 장승배기로 161(노량진동),37.512528,126.939942
3,용산구청,서울특별시 용산구 백범로79길 64(청파동3가),37.541602,126.964977
4,성동구청,서울특별시 성동구 고산자로 270(행당동),37.563423,127.036965


In [95]:
df.to_csv('서울관공서2.csv',index=False)